# Funkcje hashujące

Ćwiczenia z wykorzystaniem algorytmów kryptograficznych funkcji jednokierunkowych (hash) do ochrony bazy haseł. Poznanie zasad ataku systematycznego przeszukiwania (brute-force) na hasła.

## Wczytanie bibliotek

In [1]:
from hashlib import md5
import sys, string, crypt
import os, errno
import htpasswd
from typing import Tuple, List, Iterable
from Crypto.Cipher import ARC4
from itertools import product, takewhile
from IPython.display import clear_output

### Przydatne funkcje

In [40]:
def trivial_hash(dane):
    hash = 0
    for znak in dane:
        hash += ord(znak)
    return hash % 999

In [2]:
def strToCodes(string : str) -> Iterable[int]:
    return (ord(c) for c in string)

def codesToStr(codes : Iterable[int]) -> str:
    return ''.join((chr(c) for c in codes))

In [3]:
def display_progres(procentage_done : float, bar_length = 20) -> None:
    bar_length = 20
    fill_length = round(bar_length * procentage_done)
    
    bar = '[{}{}]'.format('#' * fill_length, '-' * (bar_length - fill_length))
    display = bar + f' {procentage_done:.0%}'
    print(display)
    
    
display_progres(0.4)

[########------------] 40%


## Przygotowanie

### Testy funkcji skrótu

Przykładowe wywołanie funkcji `crypt()` z poziomu języka Python.

In [4]:
import sys
import crypt
import string
import random
password = sys.argv[1]

salt = ''.join(random.sample(string.ascii_letters, 2))

protected_password = crypt.crypt(password, salt)
print( protected_password ) 

XJ1IYKAzNwYBs


In [5]:
random.sample(string.ascii_letters, 2)

['a', 'g']

Sprawdzanie integralności danych przy pomocy `sha256()`.

In [6]:
import sys
import hashlib
h  = hashlib.sha256( sys.argv[1].encode() )
print( h.digest() )
print( h.hexdigest() )

b'\x0ee\x03\xc1\xec\xe4\x0eN\xa7f\x84c$\x8e\xa2qn\xb3vC\xf2\xc2\xc6\x05\xf8\xbc\xeeM\x19Z\x17\x05'
0e6503c1ece40e4ea7668463248ea2716eb37643f2c2c605f8bcee4d195a1705


### Ataki na funkcje jednokierunkowe

Metoda systematycznego przeszukiwania, inaczej zwana metodą brutalnej siły (ang. brute force) to jeden z najprostszych ataków na wszelkiego rodzaju zabezpieczenia. Polega ona na ślepym przeszukiwaniu przestrzeni klucza w celu znalezienia tego właściwego. Łatwą obroną przed tego typu atakiem jest **rozszerzenie przestrzeni**, a tym samym wydłużenie czasu potrzebnego do jej przeszukania.

Jednym z najbardziej znanych programów realizujących ten atak jest _John the Ripper_. W ramach ćwiczenia będziemy korzystać ze znacznie prostszego przykładu. Przykładowy skrypt realizujący atak na hasło o długości **trzech znaków**, składające się z samych małych liter, ukryte za pomocą funkcji `crypt()`:

Ten kod **nie działa** nawet po poprawkach!?

In [7]:
import sys, string, crypt

# passwd = sys.argv[1]
# passwd = 'aba'
passwd = crypt.crypt('abc')
chars = string.ascii_letters

trail = ''
for a in chars:
    for b in chars:
        for c in chars:
            trial = a+b+c
            crypted = crypt.crypt( trial, passwd)
            if crypted == passwd:
                break
        if crypted == passwd:
            break
    if crypted == passwd:
        break
        
if crypted == passwd:
    print( "Hasło złamane: " + trail)
else:
    print( "Nie złamano hasła" )

Hasło złamane: 


### Badanie htpasswd

Program **htpasswd** dziła na serwerach Apatche http i służy do obsługi plików przechowujących nazwy użytkowników i haszhe ich haseł.  
Do przetestowania programu wykożystam kontener Dockera `httpd:2.4`.

Uruchomię go komendą `docker run -it --rm --name my-apache-app --entrypoint=bash httpd:2.4`.

Utworzyłem kilka wpisów komendami

```Bash
htpasswd -bc passwords.txt Jan 123
htpasswd -b passwords.txt Olek abc
htpasswd -bd passwords.txt Zuza 123 #Zmiana szyfru na funkcję crypt
```

Plik zawiera następującą zawartość
```
Jan:$apr1$cfj46z8f$c9BXyt1aRWXIU3LTZpD/e1
Olek:$apr1$THdv4Lr6$IpNKgVcoZosEhClnFMNH.1
Zuza:kTJQit8yheE2Y
```

**2 pierwsze znaki to sól w crypt!!**

In [7]:
crypt.crypt('123', 'kT')

'kTJQit8yheE2Y'

In [8]:
h = md5(('cfj46z8f' + '123').encode())
# h.update()
# h.update(.encode())

h.hexdigest()

'efed657a58df1a9d236672b7d596918d'

## Zadania z iSoda

### Zadanie 1

Napisz program korzystający z bazy wygenerowanej przez program _htpasswd_. Celem programu jest sprawdzenie, czy użytkownik o podanym identyfikatorze i haśle jest w pliku bazy.

#### Nieudana próba

Założe, że wszystkie hasła są haszowane domyślym algorytmem czyli **MD5**.

Liczenie haszy samemu

In [9]:
from hashlib import md5

def isUserInDataBase(file_name : str, user_name : str, password : str) -> bool:
    h = md5()
    h.update(password.encode())
    print(h.digest())

file_name = 'htpasswords.txt'
user_name = 'Jan'
password = 'abc'
isUserInDataBase(file_name, user_name, password)
# crypt.crypt('123', None)

b'\x90\x01P\x98<\xd2O\xb0\xd6\x96?}(\xe1\x7fr'


Wykożystam bibliotekę

In [10]:
import htpasswd

file_name = 'htpasswords.txt'

with htpasswd.Basic(file_name, mode="md5") as userdb:
    try:
#         userdb.add('bob', 'password')
        userdb.add('Jan', '123')
    except e:
        print (e)

FileNotFoundError: [Errno 2] No such file or directory: 'htpasswords.txt'

In [ ]:
import crypt
from hmac import compare_digest as compare_hash

plaintext = 'abc'
hashed = crypt.crypt(plaintext)
if not compare_hash(hashed, crypt.crypt(plaintext, hashed)):
    raise ValueError("hashed version doesn't validate against original")

In [ ]:
import crypt
from hmac import compare_digest as compare_hash

plaintext = 'abc'
hashed = crypt.crypt(plaintext, 'aaxx')
hashed2 = crypt.crypt(plaintext, hashed)
print(hashed)
print(hashed2)
compare_hash(hashed, hashed2)
crypt.crypt(plaintext, hashed)

In [ ]:
crypt.crypt(plaintext, 'aa')

#### Kolejne podejście

Doszedłe do wniosku, że ponowne wyliczenie hasha jest niemożliwe gdy funckcja użyta przez _htpasswd_ jest różna od `crypt()`. Jest tak ponieważ sól użyta w _MD5_ oraz _SHA_ jest różna na każdym serwerze.  
Przy metodzie `crypt()` sól jest zapisana jako pierwsze 2 znaki cryptogramu.


Żeby uzyskać taką samą sól utowrzę plik przy pomocy biblioteki [htpasswd 2.3](https://pypi.org/project/htpasswd/).

In [ ]:
htpasswd_file = 'htpasswd.txt'

try:
    os.remove(htpasswd_file)
except OSError as e: # this would be "except OSError, e:" before Python 2.6
    if e.errno != errno.ENOENT: # errno.ENOENT = no such file or directory
        raise # re-raise exception if a different error occurred
        
open(htpasswd_file, 'a').close()

users = [('Jan', '123'), ('Borhum', 'Pa$$w0rd'), ('Buck', '123')]

with htpasswd.Basic(htpasswd_file) as userdb: #mode='md5'
    for user in users:
        userdb.add(*user)
#         print(userdb._encrypt_password(user[1]))

In [ ]:
def parse_htpasswd(file_name : str) -> List[Tuple[str, str]]:
    """Read .htpasswd file and reaturn list of pairs (username, hash)"""
    results = []
    with open(file_name) as file:
        for line in file:
            results.append(tuple(line.strip().split(':')))
            
    return results
                
parse_htpasswd(htpasswd_file)

In [ ]:
def is_user_in_base(file_name : str, name : str, password : str) -> bool:
    data = parse_htpasswd(file_name)
    
    user = next((u for u in data if u[0] == name), None)
    if user == None:
        return False
    
    hash = user[1]
    salt = hash[:2]
    return crypt.crypt(password, salt) == hash
    
is_user_in_base(htpasswd_file, 'Buck', '123')

### Zadanie 2

Napisz program umożliwiający zmianę hasła użytkownika w bazie programu htpasswd.

In [ ]:
# Tworzenie pliku
htpasswd_file = 'htpasswd.txt'

try:
    os.remove(htpasswd_file)
except OSError as e: # this would be "except OSError, e:" before Python 2.6
    if e.errno != errno.ENOENT: # errno.ENOENT = no such file or directory
        raise # re-raise exception if a different error occurred
        
open(htpasswd_file, 'a').close()

users = [('Jan', '123'), ('Borhum', 'Pa$$w0rd'), ('Buck', '123')]

with htpasswd.Basic(htpasswd_file) as userdb: #mode='md5'
    for user in users:
        userdb.add(*user)
#         print(userdb._encrypt_password(user[1]))

Wykożystam bibliotekę [htpasswd 2.3](https://pypi.org/project/htpasswd/).

In [ ]:
user = 'Jan'
new_password = 'abc'

with htpasswd.Basic(htpasswd_file) as userdb:
    userdb.change_password(user, new_password)

### Zadanie 3

Napisz program umożliwiający dodawanie nowych użytkowników do bazy programu htpasswd.

In [ ]:
# Tworzenie pliku
htpasswd_file = 'htpasswd.txt'

try:
    os.remove(htpasswd_file)
except OSError as e: # this would be "except OSError, e:" before Python 2.6
    if e.errno != errno.ENOENT: # errno.ENOENT = no such file or directory
        raise # re-raise exception if a different error occurred
        
open(htpasswd_file, 'a').close()

users = [('Jan', '123'), ('Borhum', 'Pa$$w0rd'), ('Buck', '123')]

with htpasswd.Basic(htpasswd_file) as userdb: #mode='md5'
    for user in users:
        userdb.add(*user)
#         print(userdb._encrypt_password(user[1]))

Wykożystam bibliotekę [htpasswd 2.3](https://pypi.org/project/htpasswd/).

In [ ]:
def add_new_user(file_name : str, user_name : str, password : str) -> None:
    with htpasswd.Basic(htpasswd_file) as userdb: #mode='md5'
        userdb.add(user_name, password)

### Zadanie 4

Przy pomocy wywołania bibliotecznej funkcji md5() odtwórz działanie programu md5sum.

Program _md5sum_ otrzymuje listę plików jako argumenty. Dla każdego z nich wylicza sumę kontrolną i wypisuje ją na standardowe wyjście.

In [ ]:
def md5sum(file_names : Iterable[str]) -> str:
    result = []
    for file_name in file_names:
        with open(file_name, 'rb') as file:
            hash = md5(file.read()).hexdigest()
            result.append(f'{hash}  {file_name}')
            
    return '\n'.join(result)

files = [f'ExampleFile{i}.txt' for i in 'ABC']
sums = md5sum(files)
print(sums)

### Zadanie 5

Przy pomocy wywołania bibliotecznej funkcji md5() odtwórz działanie polecenia md5sum -c, czyli kontrole spójności plików.

Dla wywołania `md5sum -c` program otrzymuję listę wygenerowaną przez wywołanie `md5sum`. Następnei dla każdego pliku sprawdza czy suma kontrolna jest prawidłowa.

In [ ]:
def md5sum_c(checksum_data : str) -> str:
    result = []
    for line in checksum_data.split('\n'):
        hash, file_name = line.split('  ')
        with open(file_name, 'rb') as file:
            data = file.read()
            validation = 'OK' if hash == md5(data).hexdigest() else 'ERROR'
            result.append(f'{file_name}: {validation}')
            
    return '\n'.join(result)
            
print(md5sum_c(sums))

### Zadanie 6

Korzystając z polecania time sprawdź wydajność dowolnej implementacji algorytmu łamania haseł metodą brutalnej siły i oszacuj czas maksymalny czas potrzebny do złamania haseł różnej długości i składający się z małych liter i dużych liter.

Przetestuję algorytm łamania klucza ARC4 z laboratoriów numer 2.

In [ ]:
def entropy(data : bytes) -> float:
    count = [0] * 256
    dataSize = len(data)
    for b in data: count[b] = count[b] + 1
    
    entropy = 0
    for b in range(256):
        entropy += (count[b] / dataSize) * count[b]
        
    return 1 - entropy / dataSize


sentence = "A complete sentence must have, at minimum, three things: a subject, verb, and an object. The subject is typically a noun or a pronoun. And, if there's a subject, there's bound to be a verb because all verbs need a subject. Finally, the object of a sentence is the thing that's being acted upon by the subject."
entropy(bytes(strToCodes(sentence))) * 100

In [ ]:
def bruteforceARC4(cryptogram : bytes, keyLength : int = 3, keyAlphabet = range(256)) -> str:
    entropyThreshold = 0.9585

    possibleKeys = product(keyAlphabet, repeat=keyLength)
    for k in possibleKeys:
        k = codesToStr(k)        
        cipher = ARC4.new(k)
        recovered = cipher.decrypt(cryptogram)
        e = entropy(recovered)

        if e < entropyThreshold:
#             print(k)
            return k
#                 print('\nZnaleziono rozwiązanie!\n')
#                 print(f'Klucz: {k}')
#                 print(f'Wiadomość: {recovered}')
#                 break

    return None
#         print(f'Nie ma rozwiązania dla klucza długości {length}')

In [ ]:
data = "A complete sentence must have, at minimum, three things: a subject, verb, and an object. The subject is typically a noun or a pronoun. And, if there's a subject, there's bound to be a verb because all verbs need a subject. Finally, the object of a sentence is the thing that's being acted upon by the subject."
key = 'abc'

cipher = ARC4.new(key)
cryptogram = cipher.encrypt(data)

keyAlphabet = range(ord('a'), ord('z') + 1)
%timeit bruteforceARC4(cryptogram, 3, keyAlphabet)


In [ ]:
*_, last = takewhile(lambda x: x[1] != tuple('abc'), enumerate(product(keyAlphabet, repeat=3)))
last[0] + 1

Maksymalny czas łamania tego hasła wynosi około 2,251 ms.  
Pojedńcze złamanie hasła wymaga **17576** iteracji.

Średni czas iteracji wynosi więc 0,12807 µs.  
Moc alfabetu składającego się z dużych i małych liter wynosi 52.  
Liczba kombinacji dla hasła długości x wynosi 52^x.

### Zadanie 7

Napisz program zgadujący hasła ukryte za pomocą funkcji `crypt()`, przechowywane w bazie programu htpasswd. Przyjmij założenie, że hasła są trzy znakowe i składają się z samych małych liter.

In [41]:
def decrypt_crypt(hash : str) -> str:
    key_alphabet = list(range(ord('a'), ord('z') + 1))
    key_length = 3
    max_possibilities = len(key_alphabet) ** key_length
    
    salt = hash[:2]
    for i, plain in enumerate(product(key_alphabet, repeat=key_length)):
        plain = codesToStr(plain)
        clear_output(True)
        display_progres(i / max_possibilities)
        print(plain)
        
        if hash == crypt.crypt(plain, salt):
            clear_output(True)
            return plain
        
        
plain = 'sto'
h = crypt.crypt(plain)

recoverd = decrypt_crypt(h)
if recoverd:
    print (f'Hasło to: {recoverd}')
else:
    print('Nie udało się znaleźć hasła')

Hasło to: sto


### Zadanie 8

Przygotuj dwa dokumenty, których funkcje skrótu `trivial_hash()` bedą identyczne, a treść różna. Wykorzystaj paradoks urodzinowy.

In [13]:
def display_trivial_hash_for_file(file_name : str) -> None:
    with open(file_name) as file:
        print(trivial_hash(file.read()))

In [38]:
display_trivial_hash_for_file('TryvialDocument.txt')
display_trivial_hash_for_file('FakeTryvialDocument.txt')

2
2


### Zadanie 9

Napisz program szukający kolizji z określoną wartością funkcji skrótu `trivial_hash()` poprzez modyfikacje ‘białych’ znaków.

In [42]:
def match_hash(orginal_data : str, spoofed_data : str) -> str:
    maxTries = 100000000
    hash = trivial_hash(orginal_data)
#     Załóżmy, że nie wiem jak działa ta funkcja
    for i in range(maxTries):
        if trivial_hash(spoofed_data) == hash:
            return spoofed_data
        spoofed_data = spoofed_data + ' '
        
    raise RuntimeError(f'Maximum tires excedet - {maxTries}')

In [45]:
text = 'Tomek ma kartę'

fake = match_hash(text ,'Tomek ma długi')
print(len(fake))
print(fake)

1012
Tomek ma długi                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

### Zadnie 10

John the Ripper jest zdecydowanie szybszy.
(Nie instalowałem go, ale wszystko jest szybsze od Pythona)